# Example Seldon Core Deployments using Helm with Istio

Prequisites

 * [Install istio](https://istio.io/latest/docs/setup/getting-started/#download)

## Setup Cluster and Ingress

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Istio Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Istio). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [52]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [53]:
import time
from seldon_core.seldon_client import SeldonClient

def wait_for_seldon_ready(sc: SeldonClient, timeout: int = 60):
    start = time.time()
    while time.time() - start < timeout:
        r = sc.predict(transport="rest")
        if r.success:
            return
        time.sleep(2)
    raise TimeoutError("Seldon deployment not ready within timeout")

## Configure Istio

For this example we will create the default istio gateway for seldon which needs to be called `seldon-gateway`. You can supply your own gateway by adding to your SeldonDeployments resources the annotation `seldon.io/istio-gateway` with values the name of your istio gateway.

Create a gateway for our istio-ingress

In [54]:
%%writefile resources/seldon-gateway.yaml
apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: seldon-gateway
  namespace: istio-system
spec:
  selector:
    istio: ingressgateway # use istio default controller
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"

Overwriting resources/seldon-gateway.yaml


In [55]:
!kubectl create -f resources/seldon-gateway.yaml -n istio-system 

Error from server (AlreadyExists): error when creating "resources/seldon-gateway.yaml": gateways.networking.istio.io "seldon-gateway" already exists


Ensure the istio ingress gatewaty is port-forwarded to localhost:8004

 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8004:8080`

In [56]:
ISTIO_GATEWAY = "localhost:8004"
VERSION = !cat ../version.txt
VERSION = VERSION[0]
VERSION

'1.19.0-dev'

In [57]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

## Start Seldon Core

Use the setup notebook to [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core) with Istio Ingress. Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

## Serve Single Model

In [58]:
!helm upgrade -i mymodel ../helm-charts/seldon-single-model --set model.image=seldonio/mock_classifier:$VERSION --namespace seldon

Release "mymodel" does not exist. Installing it now.
NAME: mymodel
LAST DEPLOYED: Fri Nov 14 18:30:02 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [59]:
!kubectl wait deployment \
  -l seldon-deployment-id=mymodel \
  --for=condition=available \
  --timeout=120s

deployment.apps/mymodel-default-0-model condition met


### Get predictions

In [61]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="mymodel", namespace="seldon", gateway_endpoint=ISTIO_GATEWAY
)

wait_for_seldon_ready(sc)

#### REST Request

In [62]:
r = sc.predict(gateway="istio", transport="rest")
assert r.success == True

## gRPC Request

In [63]:
r = sc.predict(gateway="istio", transport="grpc")
assert r.success == True

In [64]:
!helm delete mymodel

release "mymodel" uninstalled


## Host Restriction

In this example we will restriction request to those with the Host header "seldon.io"

In [65]:
%%writetemplate resources/model_seldon.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: example-seldon
  annotations:
    "seldon.io/istio-host": "seldon.io"
spec:
  protocol: seldon
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: model
    replicas: 1

In [66]:
!kubectl apply -f resources/model_seldon.yaml --namespace seldon

seldondeployment.machinelearning.seldon.io/example-seldon created


In [67]:
!kubectl wait deployment \
  -l seldon-deployment-id=example-seldon \
  --for=condition=available \
  --timeout=120s

deployment.apps/example-seldon-model-0-classifier condition met


In [69]:
sc = SeldonClient(
    deployment_name="example-seldon", namespace="seldon", gateway_endpoint="localhost:8003"
)

wait_for_seldon_ready(sc)

In [70]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/example-seldon/api/v1.0/predictions \
   -H "Content-Type: application/json" \
assert X == []

In [71]:
import json
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/example-seldon/api/v1.0/predictions \
   -H "Content-Type: application/json" \
   -H "Host: seldon.io"
d=json.loads(X[0])
print(d)
assert(d["data"]["ndarray"][0][0] > 0.4)

{'data': {'names': ['proba'], 'ndarray': [[0.43782349911420193]]}, 'meta': {'requestPath': {'classifier': 'seldonio/mock_classifier:1.19.0-dev'}}}


In [72]:
!kubectl delete -f resources/model_seldon.yaml

seldondeployment.machinelearning.seldon.io "example-seldon" deleted
